# Se importan las bibliotecas necesarias

In [1]:
import pandas as pd
import re
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.linear_model import LogisticRegression
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import STOPWORDS
%matplotlib inline

# Se lee el .csv

In [2]:
train = pd.read_csv('csv/train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [3]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test = pd.read_csv('csv/test.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


# Se cargan los embeddings pre-entrenados

In [5]:
#descargar de aca https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz, pesa 1.5gb
EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin.gz'
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

# Elaboración del modelo

Para este modelo se aplicará NLP para el procesamiento de los tweets y se utilizarán distintos árboles de decisión para realizar las predicciones:

### Preparación del set de datos

In [6]:
tweets = train['text'].str.split()
clean_tweets = []

for tweet in tweets:
    clean_tweet = []
    for word in tweet:
        clean_word = re.sub('[^a-zA-Z]','', word)
        if(clean_word != '')&(clean_word not in STOPWORDS):
            clean_tweet.append(clean_word.lower())
    clean_tweets.append(clean_tweet)

In [7]:
train['clean_text'] = clean_tweets
train.set_index('id', inplace=True)
train.drop(columns=['keyword','location','text'], inplace=True)
train.head()

,target,clean_text
id,,
1,1,"[our, deeds, reason, earthquake, may, allah, f..."
4,1,"[forest, near, la, ronge, sask, canada]"
5,1,"[all, residents, asked, shelter, place, notifi..."
6,1,"[people, receive, wildfires, evacuation, order..."
7,1,"[just, got, sent, photo, ruby, alaska, smoke, ..."


In [8]:
embeddings_train = np.zeros((len(train.index),300),dtype='float32') # 300 es la longitud del embedding
normalized_embeddings_train = np.zeros((len(train.index),300),dtype='float32')

contador = 0
for tweet in clean_tweets:
    cant_palabras = 0
    embedded_tweet = np.zeros((300,), dtype='float32')
    
    for word in tweet:
        
        if word in word2vec.vocab:
            embedded_tweet = np.add(word2vec[word],embedded_tweet)
            cant_palabras += 1     
        
    embeddings_train[contador] = embedded_tweet
    
    if cant_palabras!=0:
        normalized_embeddings_train[contador] = np.divide(embedded_tweet,cant_palabras)
    else:
        normalized_embeddings_train[contador] = embedded_tweet
        
    contador += 1    

In [9]:
train['embeddings'] = embeddings_train.tolist()
train['normalized_embeddings'] = normalized_embeddings_train.tolist()

In [10]:
train.head()

,target,clean_text,embeddings,normalized_embeddings
id,,,,
1,1,"[our, deeds, reason, earthquake, may, allah, f...","[0.748291015625, 0.62420654296875, 1.207519531...","[0.1068987175822258, 0.08917236328125, 0.17250..."
4,1,"[forest, near, la, ronge, sask, canada]","[0.07080078125, -0.23956298828125, -0.13565063...","[0.0177001953125, -0.0598907470703125, -0.0339..."
5,1,"[all, residents, asked, shelter, place, notifi...","[0.127685546875, 0.06494140625, 1.075439453125...","[0.009821965359151363, 0.0049954927526414394, ..."
6,1,"[people, receive, wildfires, evacuation, order...","[1.033935546875, -0.5733642578125, 0.087036132...","[0.1723225861787796, -0.0955607071518898, 0.01..."
7,1,"[just, got, sent, photo, ruby, alaska, smoke, ...","[0.7109375, 0.258026123046875, -0.57470703125,...","[0.07109375298023224, 0.0258026123046875, -0.0..."


In [11]:
tweets = test['text'].str.split()
clean_tweets = []

for tweet in tweets:
    clean_tweet = []
    for word in tweet:
        clean_word = re.sub('[^a-zA-Z]','', word)
        if(clean_word != '')&(clean_word not in STOPWORDS):
            clean_tweet.append(clean_word.lower())
    clean_tweets.append(clean_tweet)

In [12]:
test['clean_text'] = clean_tweets
test.set_index('id', inplace=True)
test.drop(columns=['keyword','location','text'], inplace=True)
test.head()

,clean_text
id,
0,"[just, happened, terrible, car, crash]"
2,"[heard, earthquake, different, cities, stay, s..."
3,"[forest, spot, pond, geese, fleeing, street, i..."
9,"[apocalypse, lighting, spokane, wildfires]"
11,"[typhoon, soudelor, kills, china, taiwan]"


In [13]:
embeddings_test = np.zeros((len(test.index),300),dtype='float32') # 300 es la longitud del embedding
normalized_embeddings_test = np.zeros((len(test.index),300),dtype='float32')

contador = 0
for tweet in clean_tweets:
    cant_palabras = 0
    embedded_tweet = np.zeros((300,), dtype='float32')
    
    for word in tweet:
        
        if word in word2vec.vocab:
            embedded_tweet = np.add(word2vec[word],embedded_tweet)
            cant_palabras += 1     
        
    embeddings_test[contador] = embedded_tweet
    
    if cant_palabras!=0:
        normalized_embeddings_test[contador] = np.divide(embedded_tweet,cant_palabras)
    else:
        normalized_embeddings_test[contador] = embedded_tweet
        
    contador += 1    

### Entrenamiento del set de datos considerando sólo 'embeddings'

In [25]:
X, y = normalized_embeddings_train, train.iloc[:,0].to_frame()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=482)

In [26]:
xg_reg = xgb.XGBRegressor(objective ='binary:logistic', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 20, alpha = 10, n_estimators = 80)

xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

Accuracy Score : 0.8346456692913385
Precision Score : 0.8434163701067615
Recall Score : 0.7429467084639498
F1 Score : 0.79


In [27]:
df = pd.DataFrame(xg_reg.predict(X),columns=["word2vec_score"])
df.head()

,word2vec_score
0,0.726938
1,0.543444
2,0.890754
3,0.893148
4,0.891150


In [28]:
final = df.to_csv('csv/solo_embedding_word2vec_train.csv')

### Predicciones

In [30]:
test['target'] = xg_reg.predict(normalized_embeddings_test).round().astype(int)
#test.drop(columns=['clean_text'], inplace=True)
test.head()

,target
id,
0,1
2,0
3,1
9,1
11,1


In [31]:
final = test.to_csv('csv/submission_word2vec.csv')

In [32]:
df = pd.DataFrame(xg_reg.predict(normalized_embeddings_test),columns=["word2vec_score"])
df.head()
final = df.to_csv('csv/solo_embedding_word2vec_test.csv')